Written as part of https://www.scrapehero.com/how-to-scrape-amazon-product-reviews-using-python/, modified by us.

In [129]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep
from itertools import cycle 
from selenium import webdriver



In [141]:
def ParseReviews(amazon_url):
    '''
    Given a url to an amazon product, this function returns all
    of its amazon reviews in a json format.
    
    Each review is made up of both some text and a rating(1-5).
    '''

    parser = getLink(amazon_url)    
    XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
    XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
    XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'

    reviews = parser.xpath(XPATH_REVIEW_SECTION_1)

    if not reviews:
        reviews = parser.xpath(XPATH_REVIEW_SECTION_2)

    reviews_list = []


    # Parsing individual reviews
    for review in reviews:

        XPATH_RATING  = './/i[@data-hook="review-star-rating"]//text()'
        raw_review_rating = review.xpath(XPATH_RATING)
        review_rating = ''.join(raw_review_rating).replace('out of 5 stars', '')

        XPATH_REVIEW_TEXT = './/span[@data-hook="review-body"]//text()'
        raw_review_text = review.xpath(XPATH_REVIEW_TEXT)
        review_text = ' '.join(' '.join(raw_review_text).split())

        reviews_list.append({'review_text': review_text, 'review_rating': review_rating})

    data = { 'url': amazon_url,
             'reviews': reviews_list }

    return data

return {"error": "failed to process the page", "url": amazon_url}
            

NameError: name 'amazon_url' is not defined

In [99]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = get(url)
    parser = html.fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [100]:
agents = ["Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0", 
          "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36", 
          "Mozilla/5.0 (Linux; U; Android 4.4.2; en-us; SCH-I535 Build/KOT49H) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30", 
          "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322)"]
agent_pool = cycle(agents)

In [142]:
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [106]:
def isRobot(response):
    title = response.findtext('.//title')
    if "robot" in title.lower():
        print("ROBOT!")
        return True
    return False

In [135]:
def headless(amazon_url): 
    print("HEADLESS!!!")
    driver = webdriver.PhantomJS()
    driver.get(amazon_url)
    return driver.page_source


In [145]:
def getLink(amazon_url):
    headers = {'User-Agent': '''Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.96 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'''}
    response = ''
    parser = ''
    successful = False
    counter = 0
    MAX_TRIES = 10
    shuffle = False
    while not successful and counter<MAX_TRIES: 
        if not shuffle:
            response = get(amazon_url, headers = headers, timeout=30)
        else:
            agent = next(agent_pool)
            headers['User-Agent'] = agent
            try:
                print("trying proxy!")
                proxy = next(proxy_pool)    
                response = get(amazon_url,proxies={"http":proxy,"https":proxy}, headers=headers, timeout=30)
            except Exception as e:
                print("exception while trying to use a proxy", e)
        if response.status_code == 404:
            return {"url": amazon_url, "error": "page not found"}
        cleaned_response = response.text.replace('\x00', '') 
        # get html in tree structure that can be parsed with XPath
        parser = html.fromstring(cleaned_response) 
        robot = isRobot(parser)
        if robot:
            shuffle = True
        else:
            print("succcesful!")
            successful = True
        counter+=1
    if successful:
        pass
    else:
        #headless:
        response = headless(amazon_link)
        cleaned_response = response.replace('\x00','')
        parser = html.fromstring(cleaned_response)
    return parser

In [125]:
def getTotalReviewcount(amazon_url):
    '''
    Given an amazon product url, this function scrapes the total
    amount of reviews that the product has, and returns the value.
    '''
    parser = getLink(amazon_url)
    if parser:
        # getting the product name
        XPATH_PRODUCT_NAME = '//h1//span[@id="productTitle"]//text()'
        XPATH_MOBILE_NAME = '//h1[@id="title"]//text()'
        raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)
        product_name = ''.join(raw_product_name).strip()
        if  product_name: 
            raw_product_name = parser.xpath(XPATH_MOBILE_NAME)
            product_name = ''.join(raw_product_name).strip()
            if  len(product_name) == 0:
                raise Exception("PRODUCT NAME MESSED UP!")
        # getting the total review count
        XPATH_REVIEW_COUNT = '//h2[@data-hook="total-review-count"]//text()'
        raw_review_count = parser.xpath(XPATH_REVIEW_COUNT)
        if not raw_review_count:
            raise Exception("REVIEW COUNT MESSED UP!")
        review_count = raw_review_count[0].split()[0]
        review_count = int(review_count.replace(",",''))

        # getting the aggregated ratings
        XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
        total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)

        ratings_dict = {}
        for ratings in total_ratings:
            extracted_rating = ratings.xpath('./td//a//text()')
            if extracted_rating:
                rating_key = extracted_rating[0] 
                rating_value = extracted_rating[1]
                if rating_key:
                    ratings_dict.update({rating_key: rating_value})

        return { 'url': amazon_url,
                 'name': product_name,
                 'review-count': review_count,
                 'ratings': ratings_dict }
    else:
        raise Exception("Was not able to get the link")

In [134]:
getTotalReviewcount("https://www.amazon.com/Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8")

ROBOT!
trying proxy!
trying proxy!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f552214a940>: Failed to establish a new connection: [Errno 111] Connection refused',)))
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e4b278>: Failed to establish a new connection: [Errno 111] Connection refused',)))
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.ama

/home/jonny/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  import sys


{'url': 'https://www.amazon.com/Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8',
 'name': 'Echo (2nd Generation) - Smart speaker with Alexa - Charcoal Fabric',
 'review-count': 35470,
 'ratings': {'5 star': '72%',
  '4 star': '13%',
  '3 star': '6%',
  '2 star': '4%',
  '1 star': '5%'}}

In [146]:
def scrapeAmazonReviews(output_folder = None):
    '''
    Scrapes the user-inputted website for reviews, writing them
    into a json file.

    If a global path to an output folder is not inputted, this
    function defaults to writing the json file into ./scrapped-reviews/.
    '''
    if not output_folder:
        output_folder = './scrapped-reviews/'
    
    link = input('Website link:')
    
    # get the name, the amount of reviews, and the aggregated ratings
    basic_info = getTotalReviewcount(link)
    if len(basic_info['name']) > 15:
        f = open(output_folder + basic_info['name'][:15] + '..._product_reviews.json', 'w')
    else:
        f = open(output_folder + basic_info['name'][:15] + '_product_reviews.json', 'w')    
            
    dump(basic_info, f, indent=4)

    # get the reviews
    reviews_link = link.replace('/dp/', '/product-reviews/')
    print(reviews_link)
    
    last_index1 = reviews_link.find('?')
    last_index2 = reviews_link.find('/ref')
    if last_index1 is -1:
        min_index = last_index2
    elif last_index2 is -1:
        min_index = last_index1
    else:
        min_index = min(last_index1, last_index2)
        
    reviews_link = reviews_link[:min_index]
    reviews_link = reviews_link + '/?pageNumber='
    
    for i in range(1, (basic_info['review-count']//10) + 2):
        extracted_data = ParseReviews(reviews_link + str(i))
        dump(extracted_data, f, indent=4)
        
    f.close()

In [147]:
if __name__ == '__main__':
    scrapeAmazonReviews()

Website link:https://www.amazon.com/Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e750f0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/dp/B06XCM9LJ4/ref=cm_cr_arp_d_product_top?ie=UTF8 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e75710>: Failed to establish a new connection

/home/jonny/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  import sys


ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e75a58>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e4b278>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.am

succcesful!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=10 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=10 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5522342fd0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
succcesful!
succcesful

exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=28 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e4bef0>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=30 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e4b0b8>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
exception while trying to use a pro

...

succcesful!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
succcesful!
succcesful!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
succcesful!
succcesful!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1189 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521f78f60>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1189 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(104, 'Connection reset by peer')))
ROBOT

exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1204 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521f78898>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1205 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521f6e400>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
succcesful!
ROBOT!
try

ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1211 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521e50240>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1212 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5522376048>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='

succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
succcesful!
ROBOT!
trying proxy!
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1220 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5522146cf8>: Failed to establish a new connection: [Errno 111] Connection refused',)))
ROBOT!
trying proxy!
exception while trying to use a proxy HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Echo-2nd-Generation-speaker-Charcoal/product-reviews/B06XCM9LJ4/?pageNumber=1220 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f5521f52c18>: Failed to establish a new connection: [

KeyboardInterrupt: 